This file do the following things.
1. Define the model
2. Read the training and testing dataset
3. Train and test the model

### Import all neccasary modules.

In [1]:
import sys
from sklearn import preprocessing, model_selection
import tensorflow as tf
import pandas as pd
import numpy as np
import joblib
import argparse
from argparse import RawTextHelpFormatter, RawDescriptionHelpFormatter
import os
from scipy import stats
from tqdm import tqdm
import csv
from tensorflow.python.client import device_lib 
from numba import cuda
import gc
import math

### Define loss function here.
There are two loss functions: RMSE and PCC. I combined these two loss functions for training the model. 

In [2]:
def rmse(y_true, y_pred):
    dev = np.square(y_true.ravel() - y_pred.ravel())
    return np.sqrt(np.sum(dev) / y_true.shape[0])


def pcc(y_true, y_pred):
    p = stats.pearsonr(y_true, y_pred)
    if(math.isnan(p[0])): return 0.25
    return p[0]


def pcc_rmse(y_true, y_pred):

    return (1-pcc(y_true, y_pred))*alpha + rmse(y_true, y_pred) * (1 - alpha)


def PCC_RMSE(y_true, y_pred):
    global alpha

    fsp = y_pred - tf.keras.backend.mean(y_pred)
    fst = y_true - tf.keras.backend.mean(y_true)

    devP = tf.keras.backend.std(y_pred)
    devT = tf.keras.backend.std(y_true)

    r = tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1))
    r = tf.where(tf.math.is_nan(r), 1.5, r)
    
    p = tf.keras.backend.mean(fsp * fst) / (devP * devT)
    p = tf.where(tf.math.is_nan(p), 0.75, p)
    p = 1 - p
   

    return alpha * p + (1 - alpha) * r


def RMSE(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1))


def PCC(y_true, y_pred):
    fsp = y_pred - tf.keras.backend.mean(y_pred)
    fst = y_true - tf.keras.backend.mean(y_true)

    devP = tf.keras.backend.std(y_pred)
    devT = tf.keras.backend.std(y_true)

    p = tf.keras.backend.mean(fsp * fst) / (devP * devT)
    
    p = tf.where(tf.math.is_nan(p), 0.25, p)
    
    return p

### This function return a model to be trained. 
The model mostly consists of two dimentional convolution layers and dense layers.

In [3]:
def create_model(input_size, lr=0.0001, maxpool=True, dropout=0.1):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(32, kernel_size=4, strides=1,
                                     padding="valid", input_shape=input_size))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Conv2D(64, 4, 1, padding="valid"))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Conv2D(128, 4, 1, padding="valid"))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(400, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(200,
                                    kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(100, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    #model.add(tf.keras.layers.Dense(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    #model.add(tf.keras.layers.Activation("relu"))
    #model.add(tf.keras.layers.BatchNormalization())
    #model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))

    sgd = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, decay=1e-6, )
    model.compile(optimizer=sgd, loss=PCC_RMSE, metrics=['mse'])

    return model

In [4]:
reshape = [81,80,1]

### Define path files and read the input from the csv.
Note that the inputs are already preprocessed in https://github.com/Sav-eng/protein-ligand-term-project/blob/main/Generate_features.ipynb.

In [5]:
#change path
train_file = "train_test_validate_set/80_cutoffs_Ki_Kd/train.csv"
val_file = "train_test_validate_set/80_cutoffs_Ki_Kd/validate.csv"
test_file = "train_test_validate_set/80_cutoffs_Ki_Kd/test.csv"
path = ""

In [6]:
Xtrain = pd.read_csv(path + train_file,index_col=0,header = 0,names = None).dropna().values[:,1:-2]
Xtest = pd.read_csv(path + test_file,index_col=0,header = 0,names = None).dropna().values[:,1:-2]
Xval = pd.read_csv(path + val_file,index_col=0,header = 0,names = None).dropna().values[:,1:-2]

In [7]:
ytrain = []
df = pd.read_csv(path + train_file,index_col=0,header = 0,names = None).dropna()
for index,row in tqdm(df.iterrows()):
  ytrain = ytrain + [row.values[-1]]

ytest = []
df = pd.read_csv(path + test_file,index_col=0,header = 0,names = None).dropna()
for index,row in tqdm(df.iterrows()):
  ytest = ytest + [row.values[-2]]

yval = []
df = pd.read_csv(path + val_file,index_col=0,header = 0,names = None).dropna()
for index,row in tqdm(df.iterrows()):
  yval = yval + [row.values[-2]]

9656it [00:15, 607.44it/s]
350it [00:01, 327.80it/s]
1000it [00:01, 671.04it/s]


In [8]:
# Xtrain.shape

In [9]:
scaler = preprocessing.StandardScaler()
X_train_val = np.concatenate((Xtrain, Xval), axis=0)
scaler.fit(X_train_val)

StandardScaler()

In [10]:
Xtrain = scaler.transform(Xtrain).reshape(-1, reshape[0],reshape[1],reshape[2])
Xval = scaler.transform(Xval).reshape(-1, reshape[0],reshape[1],reshape[2])
Xtest = scaler.transform(Xtest).reshape(-1, reshape[0],reshape[1],reshape[2])
ytrain = np.array(ytrain).reshape(-1, 1)
yval = np.array(yval).reshape(-1, 1)
ytest = np.array(ytest).reshape(-1, 1)

In [11]:
print(device_lib.list_local_devices()) 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13807518564472134463
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4060896302102920564
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9188828800
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 18200563814209533745
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:86:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3204343632277805997
physical_device_desc: "device: XLA_GPU device"
]


In [12]:
log = []
stop = [[0,999.9], ]
batch_size = 128
epochs = 500
global alpha
alpha = 0.8
patience = 40
path_model = "models/final/"
path_log = "logs/final/"
model_name = "80_cutoffs_Ki_Kd.h5"
log_name = "80_cutoffs_Ki_Kd.csv"
delta_loss = 0.01
lr = 0.001
dropout = 0
maxpool = False
model = create_model((reshape[0], reshape[1], reshape[2]),
                                 lr=lr, dropout=dropout, maxpool=maxpool)

In [13]:
# cuda.select_device(0)
# cuda.close()

### Train and test the model

In [14]:
with open('generation.csv', mode='a') as generation_file:
    generation_writer = csv.writer(generation_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    generation_writer.writerow([model_name, log_name, lr, dropout, maxpool, alpha, batch_size])

for e in range(1, epochs+1):
    model.fit(Xtrain, ytrain, validation_data=(Xval, yval),batch_size=batch_size, epochs=1, verbose=1)

    ytrain_pred = model.predict(Xtrain).ravel()
    print(ytrain_pred)
    loss = pcc_rmse(ytrain.ravel(), ytrain_pred)
    pcc_train = pcc(ytrain.ravel(), ytrain_pred)
    rmse_train = rmse(ytrain.ravel(), ytrain_pred)

    yval_pred = model.predict(Xval).ravel()
    loss_val = pcc_rmse(yval.ravel(), yval_pred)
    pcc_val = pcc(yval.ravel(), yval_pred)
    rmse_val = rmse(yval.ravel(), yval_pred)

    ytest_pred = model.predict(Xtest).ravel()
    loss_test = pcc_rmse(ytest.ravel(), ytest_pred)
    pcc_test = pcc(ytest.ravel(), ytest_pred)
    rmse_test = rmse(ytest.ravel(), ytest_pred)

    log.append([e, loss, pcc_train, rmse_train,
                    loss_val, pcc_val, rmse_val,
                    loss_test, pcc_test, rmse_test])
    logs    = pd.DataFrame(log, columns=['epoch', 'loss', 'pcc_train', 'rmse_train',
                                             'loss_val', 'pcc_val', 'rmse_val',
                                             'loss_test', 'pcc_test', 'rmse_test'])

    print("EPOCH:%d Loss:%.3f RMSE:%.3f PCC:%.3f LOSS_VAL:%.3f RMSE_VAL:%.3f PCC_VAL:%.3f LOSS_TEST:%.3f RMSE_TEST:%.3f PCC_TEST:%.3f"%
          (e, loss, rmse_train, pcc_train, loss_val, rmse_val, pcc_val, loss_test, rmse_test, pcc_test ))            

    if(stop[-1][1] - loss_val >= delta_loss):
#         print("Model improve from %.3f to %.3f. Save model to %s."% (stop[-1][1], loss_val, path_model + model_name))
#         model.save(path_model + model_name)
        stop.append([e, loss_val])
    else:
        if(e - stop[-1][0] >= patience):
            print("Get best model at epoch = %d." % stop[-1][0])
            break
            
logs.to_csv(path_log + log_name)

76/76 [==============================] - 13s 173ms/step - loss: 13.0304 - mse: 19.1461 - val_loss: 13.3051 - val_mse: 40.8990
[0.        0.        0.        ... 0.9836105 0.2885653 0.6930298]
EPOCH:1 Loss:1.676 RMSE:6.107 PCC:0.432 LOSS_VAL:1.642 RMSE_VAL:6.395 PCC_VAL:0.547 LOSS_TEST:1.695 RMSE_TEST:6.414 PCC_TEST:0.485
76/76 [==============================] - 12s 164ms/step - loss: 12.4727 - mse: 13.7754 - val_loss: 12.7407 - val_mse: 30.9382
[0.        0.        0.592468  ... 1.9131669 1.4319898 1.5125682]
EPOCH:2 Loss:1.405 RMSE:5.314 PCC:0.572 LOSS_VAL:1.416 RMSE_VAL:5.562 PCC_VAL:0.620 LOSS_TEST:1.450 RMSE_TEST:5.623 PCC_TEST:0.594
76/76 [==============================] - 13s 165ms/step - loss: 12.0503 - mse: 12.0078 - val_loss: 12.2565 - val_mse: 24.1158
[0.        0.        1.1692796 ... 2.8493145 1.9758022 2.560081 ]
EPOCH:3 Loss:1.252 RMSE:4.722 PCC:0.616 LOSS_VAL:1.273 RMSE_VAL:4.911 PCC_VAL:0.636 LOSS_TEST:1.300 RMSE_TEST:4.958 PCC_TEST:0.615
76/76 [========================

EPOCH:26 Loss:0.379 RMSE:1.490 PCC:0.899 LOSS_VAL:0.617 RMSE_VAL:1.925 PCC_VAL:0.711 LOSS_TEST:0.644 RMSE_TEST:1.935 PCC_TEST:0.679
76/76 [==============================] - 13s 165ms/step - loss: 5.7406 - mse: 2.4038 - val_loss: 5.8887 - val_mse: 3.0596
[0.        4.370194  4.8596225 ... 9.980976  4.621877  7.367809 ]
EPOCH:27 Loss:0.365 RMSE:1.344 PCC:0.880 LOSS_VAL:0.599 RMSE_VAL:1.749 PCC_VAL:0.689 LOSS_TEST:0.603 RMSE_TEST:1.721 PCC_TEST:0.677
76/76 [==============================] - 13s 166ms/step - loss: 5.5703 - mse: 2.2403 - val_loss: 5.7293 - val_mse: 3.6006
[ 0.        4.685373  4.084828 ... 10.606867  3.654325  7.967782]
EPOCH:28 Loss:0.365 RMSE:1.452 PCC:0.906 LOSS_VAL:0.607 RMSE_VAL:1.898 PCC_VAL:0.716 LOSS_TEST:0.597 RMSE_TEST:1.791 PCC_TEST:0.702
76/76 [==============================] - 13s 165ms/step - loss: 5.4039 - mse: 2.1335 - val_loss: 5.5829 - val_mse: 3.8865
[0.        4.389107  3.5128424 ... 9.194675  4.349574  6.5814624]
EPOCH:29 Loss:0.370 RMSE:1.492 PCC:0.911

76/76 [==============================] - 13s 166ms/step - loss: 1.3251 - mse: 0.4322 - val_loss: 1.5966 - val_mse: 1.6838
[3.7072506 5.103301  4.283154  ... 8.549236  4.830148  7.0009294]
EPOCH:78 Loss:0.136 RMSE:0.544 PCC:0.966 LOSS_VAL:0.453 RMSE_VAL:1.298 PCC_VAL:0.759 LOSS_TEST:0.474 RMSE_TEST:1.348 PCC_TEST:0.744
76/76 [==============================] - 13s 165ms/step - loss: 1.2879 - mse: 0.4275 - val_loss: 1.5666 - val_mse: 1.7268
[4.0621862 5.3131227 4.227267  ... 8.828173  4.642809  7.0330143]
EPOCH:79 Loss:0.131 RMSE:0.539 PCC:0.970 LOSS_VAL:0.458 RMSE_VAL:1.314 PCC_VAL:0.757 LOSS_TEST:0.491 RMSE_TEST:1.390 PCC_TEST:0.734
76/76 [==============================] - 13s 166ms/step - loss: 1.2567 - mse: 0.4289 - val_loss: 1.5310 - val_mse: 1.6719
[3.9301195 5.387334  4.6396804 ... 8.524789  4.366781  7.5520134]
EPOCH:80 Loss:0.135 RMSE:0.539 PCC:0.966 LOSS_VAL:0.452 RMSE_VAL:1.293 PCC_VAL:0.758 LOSS_TEST:0.484 RMSE_TEST:1.366 PCC_TEST:0.736
76/76 [==============================] -

76/76 [==============================] - 13s 165ms/step - loss: 0.6406 - mse: 0.1062 - val_loss: 0.9700 - val_mse: 1.6427
[4.211195  4.7870336 4.3933773 ... 8.542194  4.6198626 7.362976 ]
EPOCH:104 Loss:0.069 RMSE:0.303 PCC:0.989 LOSS_VAL:0.450 RMSE_VAL:1.282 PCC_VAL:0.758 LOSS_TEST:0.475 RMSE_TEST:1.340 PCC_TEST:0.741
76/76 [==============================] - 13s 165ms/step - loss: 0.6257 - mse: 0.1112 - val_loss: 0.9547 - val_mse: 1.6584
[4.326152  5.416295  4.8122654 ... 8.953906  4.841205  7.344272 ]
EPOCH:105 Loss:0.075 RMSE:0.325 PCC:0.987 LOSS_VAL:0.451 RMSE_VAL:1.288 PCC_VAL:0.758 LOSS_TEST:0.476 RMSE_TEST:1.343 PCC_TEST:0.740
76/76 [==============================] - 13s 165ms/step - loss: 0.6096 - mse: 0.1105 - val_loss: 0.9288 - val_mse: 1.6070
[4.2218065 5.2464457 4.4934106 ... 8.806882  4.787158  7.2593346]
EPOCH:106 Loss:0.061 RMSE:0.275 PCC:0.992 LOSS_VAL:0.440 RMSE_VAL:1.268 PCC_VAL:0.767 LOSS_TEST:0.473 RMSE_TEST:1.341 PCC_TEST:0.744
76/76 [==============================